<a href="https://colab.research.google.com/github/sanjayssnit/langchain_basics/blob/main/langchain_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install required libraries

In [ ]:
!pip install langchain langchain-groq langchain-community

### Get API Key from Colab Secrets
This is applicable only if you are using Google Colab. In case you use Jupyter notebooks or use visual studio code, you can create a .env file and store your secrets.

In [ ]:
from google.colab import userdata
api_key = userdata.get('GROQ_API_KEY')

LLAMA_MODEL = "llama-3.3-70b-versatile"

### Chat without memory
A simple chat with LLM without using memory to store conversation history. The LLM will treat each question separately as it wont understand the previous conversations and the complete context.

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(api_key=api_key, model="llama-3.3-70b-versatile")

def simple_chat_without_memory(user_query):
  response = llm.invoke(user_query)
  return response.content

In [ ]:
response1 = simple_chat_without_memory("I want to learn AI")
print(response1)

In [ ]:
response2 = simple_chat_without_memory("Can you give me a learning plan?")
print(response2)

### Chat with memory managed using internal List
A simple chat with LLM with basic memory management to store conversation history. The LLM will read each question along with the previous conversation and answer based on the complete context.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

conversation = []

def simple_chat_with_memory(user_query):
  conversation.append(HumanMessage(content=user_query))
  response = llm.invoke(conversation)
  conversation.append(AIMessage(content=response.content))
  return response.content

In [ ]:
response1 = simple_chat_with_memory("I want to learn AI")
print(response1)

In [ ]:
print(conversation)

In [ ]:
response2 = simple_chat_with_memory("Can you give me a learning plan?")
print(response2)

### Chat prompt template and Chaining
Langchain's chaining concept using Chat prompt template and an LLM invocation

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(messages=[
    ("system", "You are a helpful personal assistant"),
    ("human", "{user_query}")
])

chain = prompt | llm
response_chain = chain.invoke({"user_query":"I have decided to learn AI"})
print(response_chain)

In [ ]:
response_chain = chain.invoke({"user_query":"Can you give me a learning plan?"})
print(response_chain)

### Cahining with memory managed in prompt template

In [ ]:
prompt_with_memory = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful personal assistant"),
    ("placeholder", "{chat_history}"),
    ("human", "{user_query}")
])

chain_with_memory = prompt_with_memory | llm

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
# Function to get user's conversation history based on the user's session id
def get_history(session_id: str):
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

In [ ]:
from langchain_core.runnables import RunnableWithMessageHistory

# Runnable with message history enables us to run a specified chain and
# gets the user's history using a callback function.

chat_with_memory = RunnableWithMessageHistory(
    runnable=chain_with_memory,
    get_session_history=get_history,
    input_messages_key="user_query",
    history_messages_key="chat_history"
)

response = chat_with_memory.invoke({"user_query":"I want to learn AI"}, {"configurable": {"session_id": "user1"}})
print(response)

In [ ]:
response2 = chat_with_memory.invoke({"user_query":"Can you give me a learning plan"}, {"configurable": {"session_id": "user1"}})
print(response2)

In [ ]:
# Question from a different user is get's conversation history only for that user.
response3 = chat_with_memory.invoke({"user_query":"Can you give me a learning plan"}, {"configurable": {"session_id": "user2"}})
print(response3)